In [2]:
from IPython.display import display
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

dir = "./csv_results/final_results/"



def calculate_stats(path, alpha):
    data = pd.read_csv(path)
    df = pd.DataFrame(data)
    # Calculate mean and std for 'our' method for each alpha
    alphas = df['alpha'].dropna().unique()
    new_df_list = []

    for alpha_ in alphas:
        our_stats = df[(df['method'] == 'our') & (df['alpha'] == alpha_)][['mia', 'forget', 'retain', 'time']].agg(['mean', 'std'])
        retrained_stats = df[df['method'] == 'retrained'][['mia', 'forget', 'retain', 'time']].agg(['mean', 'std'])

        # Calculate absolute differences
        abs_diff = (our_stats.loc['mean'] - retrained_stats.loc['mean']).abs()

        # Create new dataframe
        new_df = pd.DataFrame({
            'method': ['our'],
            'alpha': [alpha_],
            'mia': [f"{our_stats.loc['mean', 'mia']:.2f} ± {our_stats.loc['std', 'mia']:.2f} ({abs_diff['mia']:.2f})"],
            'forget': [f"{our_stats.loc['mean', 'forget']:.2f} ± {our_stats.loc['std', 'forget']:.2f} ({abs_diff['forget']:.2f})"],
            'retain': [f"{our_stats.loc['mean', 'retain']:.2f} ± {our_stats.loc['std', 'retain']:.2f} ({abs_diff['retain']:.2f})"],
            'time': [f"{our_stats.loc['mean', 'time']:.2f} ± {our_stats.loc['std', 'time']:.2f}"],
            'avg_gap': [round(abs_diff[['mia', 'forget', 'retain']].mean(), 3)]
        })

        new_df_list.append(new_df)

    # Concatenate all the new dataframes
    our_final_df = pd.concat(new_df_list, ignore_index=True)

    # Calculate mean and std for remaining methods without alphas
    methods_without_alpha = df['method'].dropna().unique()
    methods_without_alpha = [method for method in methods_without_alpha if method != 'our']

    new_df_list = []
    for method in methods_without_alpha:
        method_stats = df[df['method'] == method][['mia', 'forget', 'retain', 'time']].agg(['mean', 'std'])
        retrained_stats = df[df['method'] == 'retrained'][['mia', 'forget', 'retain', 'time']].agg(['mean', 'std'])

        # Calculate absolute differences
        abs_diff = (method_stats.loc['mean'] - retrained_stats.loc['mean']).abs()

        # Create new dataframe
        new_df = pd.DataFrame({
            'method': [method],
            'alpha': [None],
            'mia': [f"{method_stats.loc['mean', 'mia']:.2f} ± {method_stats.loc['std', 'mia']:.2f} ({abs_diff['mia']:.2f})"],
            'forget': [f"{method_stats.loc['mean', 'forget']:.2f} ± {method_stats.loc['std', 'forget']:.2f} ({abs_diff['forget']:.2f})"],
            'retain': [f"{method_stats.loc['mean', 'retain']:.2f} ± {method_stats.loc['std', 'retain']:.2f} ({abs_diff['retain']:.2f})"],
            'time': [f"{method_stats.loc['mean', 'time']:.2f} ± {method_stats.loc['std', 'time']:.2f}"],
            'avg_gap': [round(abs_diff[['mia', 'forget', 'retain']].mean(), 3)]
        })

        new_df_list.append(new_df)
    new_df_list.append(our_final_df[our_final_df['alpha']==alpha])

    # Concatenate all the new dataframes
    rest_final_df = pd.concat(new_df_list, ignore_index=True)
    
    # Rank both dataframes separately by avg_gap
    our_final_df = our_final_df.sort_values(by='avg_gap', ascending=True).reset_index(drop=True)
    rest_final_df = rest_final_df.sort_values(by='avg_gap', ascending=True).reset_index(drop=True)
    
    return our_final_df, rest_final_df

Instance-wise Unlearning with ViT

In [3]:
# ViT CIFAR-10 Instance-wise
our, rest = calculate_stats(dir + "vit-instances/vit-cifar-10.csv", alpha=8)
rest.drop(columns=['alpha'], inplace=True)
rest

,method,mia,forget,retain,time,avg_gap
0,retrained,0.88 ± 0.01 (0.00),0.99 ± 0.00 (0.00),1.00 ± 0.00 (0.00),76.54 ± 6.66,0.000
1,original,0.88 ± 0.00 (0.00),1.00 ± 0.00 (0.01),1.00 ± 0.00 (0.00),84.29 ± 9.71,0.004
2,scrub,0.88 ± 0.01 (0.01),1.00 ± 0.00 (0.01),1.00 ± 0.00 (0.00),16.81 ± 0.36,0.006
3,our,0.87 ± 0.00 (0.01),1.00 ± 0.00 (0.01),1.00 ± 0.00 (0.00),7.03 ± 0.03,0.006
4,ssd,0.89 ± 0.00 (0.01),1.00 ± 0.00 (0.01),1.00 ± 0.00 (0.00),13.70 ± 0.25,0.008
5,finetune,0.90 ± 0.01 (0.03),0.99 ± 0.01 (0.00),1.00 ± 0.00 (0.00),11.36 ± 0.24,0.010
6,unsir,0.90 ± 0.00 (0.02),0.99 ± 0.01 (0.00),0.99 ± 0.01 (0.01),10.72 ± 0.21,0.011
7,amnesiac,0.83 ± 0.01 (0.04),0.99 ± 0.00 (0.00),1.00 ± 0.00 (0.00),12.87 ± 0.25,0.014
8,neggrad,0.91 ± 0.00 (0.03),1.00 ± 0.00 (0.01),1.00 ± 0.00 (0.00),12.66 ± 0.27,0.014
9,bad-teacher,0.80 ± 0.01 (0.07),0.97 ± 0.00 (0.02),0.97 ± 0.00 (0.03),8.19 ± 0.11,0.041


In [4]:
# ViT CIFAR-100 Instance-wise
our, rest = calculate_stats(dir + "vit-instances/vit-cifar-100.csv", alpha=8)
rest.drop(columns=['alpha'], inplace=True)
rest

,method,mia,forget,retain,time,avg_gap
0,retrained,0.72 ± 0.00 (0.00),0.92 ± 0.00 (0.00),0.96 ± 0.00 (0.00),115.08 ± 3.86,0.000
1,our,0.71 ± 0.00 (0.01),0.96 ± 0.00 (0.04),0.96 ± 0.00 (0.00),7.05 ± 0.00,0.017
2,amnesiac,0.75 ± 0.01 (0.03),0.94 ± 0.01 (0.02),0.98 ± 0.01 (0.02),13.01 ± 0.47,0.022
3,finetune,0.76 ± 0.02 (0.04),0.94 ± 0.02 (0.02),0.97 ± 0.01 (0.01),11.40 ± 0.18,0.023
4,ssd,0.75 ± 0.00 (0.03),0.97 ± 0.01 (0.05),0.96 ± 0.00 (0.00),13.83 ± 0.37,0.026
5,scrub,0.75 ± 0.00 (0.03),0.97 ± 0.00 (0.05),0.96 ± 0.00 (0.00),16.92 ± 0.13,0.027
6,original,0.75 ± 0.00 (0.03),0.97 ± 0.00 (0.05),0.96 ± 0.00 (0.00),122.19 ± 1.45,0.027
7,unsir,0.78 ± 0.00 (0.06),0.94 ± 0.01 (0.02),0.95 ± 0.01 (0.01),10.75 ± 0.20,0.032
8,bad-teacher,0.65 ± 0.01 (0.07),0.90 ± 0.01 (0.02),0.91 ± 0.01 (0.05),8.05 ± 0.08,0.046
9,neggrad,0.81 ± 0.01 (0.09),0.98 ± 0.00 (0.06),0.98 ± 0.00 (0.02),12.79 ± 0.37,0.056


In [5]:
# Vit MUFAC
our, rest = calculate_stats(dir + "vit-instances/vit-mufac.csv", alpha=8)
rest.drop(columns=['alpha'], inplace=True)
rest

,method,mia,forget,retain,time,avg_gap
0,retrained,0.51 ± 0.10 (0.00),0.57 ± 0.01 (0.00),0.71 ± 0.09 (0.00),10.61 ± 5.98,0.000
1,amnesiac,0.54 ± 0.11 (0.04),0.60 ± 0.06 (0.03),0.70 ± 0.05 (0.01),1.78 ± 0.02,0.026
2,unsir,0.49 ± 0.09 (0.02),0.63 ± 0.09 (0.05),0.78 ± 0.09 (0.07),3.22 ± 0.02,0.047
3,ssd,0.55 ± 0.08 (0.04),0.66 ± 0.07 (0.09),0.69 ± 0.06 (0.01),1.94 ± 0.02,0.048
4,our,0.55 ± 0.07 (0.04),0.68 ± 0.12 (0.11),0.72 ± 0.10 (0.01),1.10 ± 0.01,0.054
5,finetune,0.54 ± 0.07 (0.03),0.66 ± 0.11 (0.09),0.77 ± 0.11 (0.07),1.41 ± 0.01,0.061
6,original,0.55 ± 0.05 (0.05),0.71 ± 0.13 (0.13),0.72 ± 0.12 (0.02),13.23 ± 6.22,0.066
7,scrub,0.55 ± 0.06 (0.04),0.71 ± 0.13 (0.13),0.73 ± 0.11 (0.02),2.23 ± 0.02,0.067
8,bad-teacher,0.54 ± 0.10 (0.03),0.51 ± 0.02 (0.07),0.56 ± 0.03 (0.15),1.20 ± 0.02,0.081
9,neggrad,0.55 ± 0.05 (0.04),0.74 ± 0.13 (0.17),0.75 ± 0.12 (0.04),1.70 ± 0.02,0.084


Instance-wise Unlearning with ResNet18

In [6]:
# ResNet18 CIFAR-10
our, rest = calculate_stats(dir + "resnet-instances/resnet-cifar10.csv", alpha=512)
rest.drop(columns=['alpha'], inplace=True)
rest

,method,mia,forget,retain,time,avg_gap
0,retrained,0.76 ± 0.03 (0.00),0.92 ± 0.02 (0.00),0.99 ± 0.02 (0.00),4.46 ± 2.17,0.000
1,our,0.67 ± 0.58 (0.10),0.96 ± 0.01 (0.04),0.98 ± 0.00 (0.01),0.30 ± 0.00,0.048
2,original,0.94 ± 0.00 (0.18),1.00 ± 0.00 (0.08),1.00 ± 0.00 (0.01),6.56 ± 0.40,0.091
3,scrub,0.94 ± 0.01 (0.18),1.00 ± 0.00 (0.08),1.00 ± 0.00 (0.01),0.58 ± 0.01,0.092
4,bad-teacher,0.48 ± 0.21 (0.29),0.99 ± 0.00 (0.07),0.99 ± 0.01 (0.00),0.30 ± 0.00,0.120
5,ssd,0.75 ± 0.27 (0.01),0.77 ± 0.38 (0.14),0.77 ± 0.38 (0.21),0.54 ± 0.00,0.123
6,unsir,0.46 ± 0.01 (0.31),0.99 ± 0.01 (0.08),0.99 ± 0.00 (0.00),0.46 ± 0.02,0.129
7,neggrad,0.45 ± 0.01 (0.32),1.00 ± 0.00 (0.08),1.00 ± 0.00 (0.01),0.49 ± 0.01,0.138
8,finetune,0.15 ± 0.25 (0.62),1.00 ± 0.00 (0.08),1.00 ± 0.00 (0.01),0.42 ± 0.01,0.238
9,amnesiac,0.00 ± 0.00 (0.76),0.98 ± 0.02 (0.06),1.00 ± 0.00 (0.01),0.57 ± 0.01,0.280


In [7]:
# ResNet18 CIFAR-100
our, rest = calculate_stats(dir + "resnet-instances/resnet-cifar100.csv", alpha=512)
rest.drop(columns=['alpha'], inplace=True)
rest

,method,mia,forget,retain,time,avg_gap
0,retrained,0.60 ± 0.17 (0.00),0.71 ± 0.19 (0.00),0.96 ± 0.04 (0.00),2.88 ± 2.58,0.000
1,our,0.67 ± 0.58 (0.07),0.75 ± 0.19 (0.04),0.92 ± 0.06 (0.04),0.30 ± 0.00,0.049
2,ssd,0.72 ± 0.17 (0.12),0.80 ± 0.17 (0.09),0.80 ± 0.17 (0.17),0.55 ± 0.02,0.126
3,scrub,0.83 ± 0.11 (0.23),0.95 ± 0.06 (0.24),0.95 ± 0.06 (0.02),0.60 ± 0.03,0.162
4,original,0.82 ± 0.11 (0.23),0.95 ± 0.05 (0.24),0.95 ± 0.06 (0.02),3.24 ± 2.60,0.162
5,bad-teacher,0.22 ± 0.39 (0.37),0.96 ± 0.04 (0.25),0.94 ± 0.05 (0.02),0.31 ± 0.01,0.213
6,unsir,0.15 ± 0.26 (0.45),0.98 ± 0.01 (0.27),0.98 ± 0.01 (0.02),0.46 ± 0.02,0.246
7,neggrad,0.15 ± 0.26 (0.45),1.00 ± 0.00 (0.29),1.00 ± 0.00 (0.04),0.50 ± 0.02,0.258
8,amnesiac,0.00 ± 0.00 (0.60),0.98 ± 0.02 (0.27),1.00 ± 0.00 (0.04),0.58 ± 0.02,0.301
9,finetune,0.00 ± 0.00 (0.60),0.99 ± 0.01 (0.28),1.00 ± 0.00 (0.04),0.43 ± 0.02,0.306


In [8]:
# ResNet18 MUFAC 
our, rest = calculate_stats(dir + "resnet-instances/resnet-mufac.csv", alpha=512)
rest.drop(columns=['alpha'], inplace=True)
rest

,method,mia,forget,retain,time,avg_gap
0,retrained,0.49 ± 0.03 (0.00),0.47 ± 0.03 (0.00),0.89 ± 0.04 (0.00),3.11 ± 0.79,0.000
1,our,0.46 ± 0.10 (0.03),0.65 ± 0.02 (0.18),0.83 ± 0.03 (0.07),0.59 ± 0.00,0.091
2,bad-teacher,0.32 ± 0.01 (0.17),0.66 ± 0.02 (0.19),0.84 ± 0.04 (0.05),0.63 ± 0.00,0.137
3,ssd,0.68 ± 0.05 (0.20),0.87 ± 0.07 (0.40),0.87 ± 0.06 (0.02),1.09 ± 0.00,0.206
4,original,0.69 ± 0.05 (0.20),0.88 ± 0.05 (0.41),0.89 ± 0.04 (0.00),3.72 ± 0.78,0.206
5,scrub,0.70 ± 0.04 (0.21),0.88 ± 0.05 (0.41),0.88 ± 0.04 (0.01),1.22 ± 0.00,0.210
6,finetune,0.54 ± 0.10 (0.06),0.98 ± 0.01 (0.51),0.99 ± 0.00 (0.10),0.77 ± 0.00,0.221
7,neggrad,0.54 ± 0.09 (0.06),0.99 ± 0.01 (0.52),0.99 ± 0.00 (0.10),0.92 ± 0.00,0.226
8,amnesiac,0.66 ± 0.31 (0.17),0.90 ± 0.01 (0.43),0.98 ± 0.01 (0.09),1.08 ± 0.00,0.230
9,unsir,0.73 ± 0.23 (0.24),0.97 ± 0.02 (0.50),0.98 ± 0.01 (0.09),1.77 ± 0.01,0.278
